In [158]:
from os import environ as env
import boto3
import yfinance as yf
import pandas as pd

In [163]:
def calc_stock(high, current):
    """
    :param high: float
    :param current: float
    :return: ratio: float
    """
    ratio = round(((current - high) / high) * 100, 2)
    return ratio


def convert_tuple(tup):
    """
    :param tup: tuple: tuple containing ranked pairs
    :return: string_tup: str: stringified version of incoming tuple
    """
    string_tup = f"{tup[0]} : {tup[1]}"
    return string_tup


def create_message(pairs, mode='personal'):
    """
    :param pairs: dict: contains ranked pairs
    :return: message: str: string of ranked pairs
    """
    message = f"\n\n{mode.upper()} ORDERED RATIOS:\n\n"
    for pair in pairs:
        message += convert_tuple(pair) + "\n"
    return message


def publish_message_sns(message):
    """
    :param message: str: message to be sent to SNS
    :return: None
    """
    sns_arn = env.get('SNS_ARN').strip()
    sns_client = boto3.client('sns')
    try:
        response = sns_client.publish(
            TopicArn=sns_arn,
            Message=message
        )

    except Exception as e:
        print(f"ERROR PUBLISHING MESSAGE TO SNS: {e}")


def get_data(tickers_list, period):
    """
    :param tickers: str: stock ticker string
    :param period: str: valid date period for comparison
    :return: temp_string, delta: str, float: stock printing statements and ratio are returned
    """
    pairs = dict()
    temp_string = ""
    tickers = " ".join([x.upper() for x in tickers_list]).strip()
    stocks = yf.Tickers(tickers)
    data = stocks.history(env.get('PERIOD', period))['Close']
    
    for ticker in tickers_list:
        try:
            df = data[ticker]
            df.dropna(inplace=True)
            close = df[-1]
            close_date = df.index[-1]
            temp_string += f"{ticker} Close {close_date.strftime('%Y-%m-%d')}: {close:.2f}\n"

            high = max(df)
            temp_string += f"{ticker} {env.get('PERIOD', period)}-High: {high:.2f}\n"

            delta = calc_stock(high, close)
            pairs[ticker] = delta

            temp_string += f"{ticker} Delta: {delta}\n\n"
        except KeyError as ke:
            print(f"Couldn't find {ticker} in data")

    return temp_string, pairs


def read_tickers(mode='period', period='5y'):
    """
    :param mode: str: personal will use personal_portfolio_stock_tickers.txt. Any other mode will simply use the S&P500
    :param period: str: valid period.
    :return: out_string,sorted(pairs.items(), key=lambda x: x[1]): str, list: string for message and sorted dict in list
    """
    out_string = "\n\nPERSONAL PORTFOLIO INDIVIDUAL HOLDING STATS:\n\n"

    if mode == 'personal':
        tickers_list = []
        print(f"\nRunning program on personal portfolio with period {period}...\n")
        with open('deployment/personal_portfolio_stock_tickers.txt', 'r') as f:
            while True:
                ticker = (f.readline()).strip()
                if ticker == "":
                    break
                tickers_list.append(ticker)
                if not ticker:
                    break

            try:
                temp_string, pairs = get_data(tickers_list, period)
                out_string += temp_string

            except Exception as e:
                print(e)
                print(f"ERROR WITH TICKER {ticker}: {e}")
                
    else:
        print(f"\nRunning program on full S&P with period {period}...\n")
        table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
        df = table[0]
        df['Symbol'] = df['Symbol'].str.replace('.','')
        tickers_list = [x for x in df.Symbol]
        
        try:
            temp_string, pairs = get_data(tickers_list, period)
            out_string += temp_string

        except Exception as e:
            print(e)
            print(f"ERROR WITH TICKER {ticker}: {e}")
    
    
    print(out_string)
    return out_string, sorted(pairs.items(), key=lambda x: x[1])


# def handler(event, context):
#     """
#     This function drives the AWS lambda. Requires 1 env var to work correctly: SNS_TOPIC which represents the topic arn
#     to which you want to publish.
#     """
#     out_string, pairs = read_tickers()
#     message = create_message(pairs)
#     message += out_string
#     print(message)
#     publish_message_sns(message)
#     return message


def handler():
    """
    This function drives the AWS lambda. Requires 1 env var to work correctly: SNS_TOPIC which represents the topic arn to which
    you want to publish. 
    """
    personal_string, personal_pairs = read_tickers(mode='personal', period='5y')
    message = create_message(personal_pairs, mode='personal')
    message += personal_string + "\n\n––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––\n\n"
    
    snp_string, snp_pairs = read_tickers(mode='S&P', period='5y')
    message += create_message(snp_pairs, 'S&P')
    
    print(message)
    
#     publish_message_sns(message)


In [ ]:
handler()


Running program on personal portfolio with period 5y...

[*********************100%***********************]  59 of 59 completed


In [132]:
stocks = yf.Tickers("AAPL MSFT")
data = stocks.history("5d")
print(data)

[*********************100%***********************]  2 of 2 completed
                 Close             Dividends             High              \
                  AAPL        MSFT      AAPL MSFT        AAPL        MSFT   
Date                                                                        
2020-04-16  286.690002  177.039993         0    0  288.200012  177.279999   
2020-04-17  282.799988  178.600006         0    0  286.950012  180.000000   
2020-04-20  276.929993  175.059998         0    0  281.679993  178.750000   
2020-04-21  268.369995  167.820007         0    0  277.250000  173.669998   
2020-04-22  276.100006  173.520004         0    0  277.850006  174.000000   

                   Low                    Open             Stock Splits       \
                  AAPL        MSFT        AAPL        MSFT         AAPL MSFT   
Date                                                                           
2020-04-16  282.350006  172.899994  287.380005  174.300003            0   

In [136]:
data[data['Close'].notnull()]

ValueError: cannot join with no overlapping index names

In [147]:
data

Close             Dividends             High              \
                  AAPL        MSFT      AAPL MSFT        AAPL        MSFT   
Date                                                                        
2020-04-16  286.690002  177.039993         0    0  288.200012  177.279999   
2020-04-17  282.799988  178.600006         0    0  286.950012  180.000000   
2020-04-20  276.929993  175.059998         0    0  281.679993  178.750000   
2020-04-21  268.369995  167.820007         0    0  277.250000  173.669998   
2020-04-22  276.100006  173.520004         0    0  277.850006  174.000000   

                   Low                    Open             Stock Splits       \
                  AAPL        MSFT        AAPL        MSFT         AAPL MSFT   
Date                                                                           
2020-04-16  282.350006  172.899994  287.380005  174.300003            0    0   
2020-04-17  276.859985  175.869995  284.690002  179.500000            0    0   
2020-04-20  276.850006  174.990005  277.950012  176.630005            0    0   
2020-04-21  265.429993  166.110001  276.279999  173.500000            0    0   
2020-04-22  272.220001  170.830002  273.609985  171.389999            0    0   

              Volume            
                AAPL      MSFT  
Date                            
2020-04-16  39281300  50479600  
2020-04-17  53812500  52765600  
2020-04-20  32503800  36669600  
2020-04-21  45189800  56131400  
2020-04-22  27955460  32736964

In [141]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
df['Symbol'] = df['Symbol'].str.replace('.','')
tickers_list = [x for x in df.Symbol]

In [142]:
tickers_list

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'AGN',
 'ADS',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BDX',
 'BRKB',
 'BBY',
 'BIIB',
 'BLK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BFB',
 'CHRW',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CPRI',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CMA',